In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import scipy.io

In [2]:
# Define the LEMCell
class LEMCell(nn.Module):
    def __init__(self, ninp, nhid, dt):
        super(LEMCell, self).__init__()
        self.ninp = ninp
        self.nhid = nhid
        self.dt = dt
        self.inp2hid = nn.Linear(ninp, 4 * nhid)
        self.hid2hid = nn.Linear(nhid, 3 * nhid)
        self.transform_z = nn.Linear(nhid, nhid)
        self.reset_parameters()

    def reset_parameters(self):
        std = 1.0 / np.sqrt(self.nhid)
        for w in self.parameters():
            w.data.uniform_(-std, std)

    def forward(self, x, y, z):
        transformed_inp = self.inp2hid(x)
        transformed_hid = self.hid2hid(y)
        i_dt1, i_dt2, i_z, i_y = transformed_inp.chunk(4, 1)
        h_dt1, h_dt2, h_y = transformed_hid.chunk(3, 1)

        ms_dt_bar = self.dt * torch.sigmoid(i_dt1 + h_dt1)
        ms_dt = self.dt * torch.sigmoid(i_dt2 + h_dt2)

        z = (1. - ms_dt) * z + ms_dt * torch.tanh(i_y + h_y)
        y = (1. - ms_dt_bar) * y + ms_dt_bar * torch.tanh(self.transform_z(z) + i_z)

        return y, z

# Define the LEM model
class LEM(nn.Module):
    def __init__(self, ninp, nhid, nout, dt=1.):
        super(LEM, self).__init__()
        self.nhid = nhid
        self.cell = LEMCell(ninp, nhid, dt)
        self.classifier = nn.Linear(nhid, nout)
        self.init_weights()

    def init_weights(self):
        for name, param in self.named_parameters():
            if 'classifier' in name and 'weight' in name:
                nn.init.kaiming_normal_(param.data)

    def forward(self, input):
        y = input.data.new(input.size(1), self.nhid).zero_()
        z = input.data.new(input.size(1), self.nhid).zero_()
        for x in input:
            y, z = self.cell(x, y, z)
        out = self.classifier(y)
        return out


### PINN data importing

In [3]:
# importing data

# Load the .mat file
mat_data = scipy.io.loadmat('burg.mat')

# Access the variables stored in the .mat file
# The variable names in the .mat file become keys in the loaded dictionary
x = mat_data['x']
t = mat_data['t']
u = mat_data['u1']



### Exact Solution data importing

In [4]:
# importing data

import torch
import torch.nn as nn
import numpy as np
import time
import scipy.io

# Load the .mat file
mat_data = scipy.io.loadmat('burgers_shock.mat')

# Access the variables stored in the .mat file
# The variable names in the .mat file become keys in the loaded dictionary
x = mat_data['x']
t = mat_data['t']
u_1 = mat_data['usol']


In [5]:
# Set random seed for reproducibility
#torch.manual_seed(42)

# Toy problem data
input_size = 256
hidden_size = 32
output_size = 256
sequence_length = 79
batch_size = 1
num_epochs = 20000

# Set random seed for reproducibility
#torch.manual_seed(42)
u[:, 0:100].shape

(256, 100)

In [6]:
input_data = u[:,0:79]
target_data = u[:,1:80]

test_data = u[:,79]
#test_target = u[:,80:100]

print("test data shape", test_data.shape)
#print("test target shape", test_target.shape)

print("input data shape",input_data.shape)
print("Target data shape",target_data.shape)

# Convert data to tensors
input_tensor = torch.tensor(input_data.T).view(batch_size, sequence_length, input_size).float()
target_tensor = torch.tensor(target_data.T).view(batch_size, sequence_length, output_size).float()

print("input tensor shape",input_tensor.shape)
print("Target tensor shape",target_tensor.shape)

test data shape (256,)
input data shape (256, 79)
Target data shape (256, 79)
input tensor shape torch.Size([1, 79, 256])
Target tensor shape torch.Size([1, 79, 256])


In [7]:
# Convert test data to tensors
test_tensor = torch.tensor(test_data.T).view(batch_size, 1, input_size).float()
#test_target_tensor = torch.tensor(test_target.T).view(batch_size, 20, output_size).float()
target_tensor = torch.squeeze(target_tensor)

In [8]:
# Create LEM instance
lem = LEM(input_size, hidden_size, output_size, dt=0.7)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(lem.parameters(), lr=0.001)

# Training loop
for epoch in range(num_epochs):
    # Forward pass
    output = lem(input_tensor)
    loss = criterion(output, target_tensor)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print progress
    if (epoch + 1) % 10 == 0:
        print(f'Epoch: {epoch + 1}/{num_epochs}, Loss: {loss.item():.16f}')

Epoch: 10/20000, Loss: 0.3632635772228241
Epoch: 20/20000, Loss: 0.3041091263294220
Epoch: 30/20000, Loss: 0.2450173050165176
Epoch: 40/20000, Loss: 0.1882142722606659
Epoch: 50/20000, Loss: 0.1358413994312286
Epoch: 60/20000, Loss: 0.0920492634177208
Epoch: 70/20000, Loss: 0.0604817606508732
Epoch: 80/20000, Loss: 0.0399516820907593
Epoch: 90/20000, Loss: 0.0275592524558306
Epoch: 100/20000, Loss: 0.0203636903315783
Epoch: 110/20000, Loss: 0.0163965262472630
Epoch: 120/20000, Loss: 0.0141055071726441
Epoch: 130/20000, Loss: 0.0125281428918242
Epoch: 140/20000, Loss: 0.0112330699339509
Epoch: 150/20000, Loss: 0.0100468685850501
Epoch: 160/20000, Loss: 0.0089271487668157
Epoch: 170/20000, Loss: 0.0078733600676060
Epoch: 180/20000, Loss: 0.0068998616188765
Epoch: 190/20000, Loss: 0.0060198819264770
Epoch: 200/20000, Loss: 0.0052409833297133
Epoch: 210/20000, Loss: 0.0045633604750037
Epoch: 220/20000, Loss: 0.0039836396463215
Epoch: 230/20000, Loss: 0.0034939160104841
Epoch: 240/20000, Lo

Epoch: 2000/20000, Loss: 0.0001114078986575
Epoch: 2010/20000, Loss: 0.0001359580346616
Epoch: 2020/20000, Loss: 0.0001294253306696
Epoch: 2030/20000, Loss: 0.0001121178574977
Epoch: 2040/20000, Loss: 0.0001106857089326
Epoch: 2050/20000, Loss: 0.0001081952796085
Epoch: 2060/20000, Loss: 0.0001064573225449
Epoch: 2070/20000, Loss: 0.0001056891487679
Epoch: 2080/20000, Loss: 0.0001048869162332
Epoch: 2090/20000, Loss: 0.0001042025251081
Epoch: 2100/20000, Loss: 0.0001035063978634
Epoch: 2110/20000, Loss: 0.0001028442638926
Epoch: 2120/20000, Loss: 0.0001022004362312
Epoch: 2130/20000, Loss: 0.0001015739762806
Epoch: 2140/20000, Loss: 0.0001009629122564
Epoch: 2150/20000, Loss: 0.0001003666984616
Epoch: 2160/20000, Loss: 0.0000997847382678
Epoch: 2170/20000, Loss: 0.0000992163622868
Epoch: 2180/20000, Loss: 0.0000986614832073
Epoch: 2190/20000, Loss: 0.0000981455086730
Epoch: 2200/20000, Loss: 0.0001012886859826
Epoch: 2210/20000, Loss: 0.0001475926255807
Epoch: 2220/20000, Loss: 0.00011

Epoch: 4030/20000, Loss: 0.0000375493182219
Epoch: 4040/20000, Loss: 0.0000371440692106
Epoch: 4050/20000, Loss: 0.0000368275759683
Epoch: 4060/20000, Loss: 0.0000365244559362
Epoch: 4070/20000, Loss: 0.0000362856626452
Epoch: 4080/20000, Loss: 0.0000379512166546
Epoch: 4090/20000, Loss: 0.0001105090632336
Epoch: 4100/20000, Loss: 0.0000442946366093
Epoch: 4110/20000, Loss: 0.0000403256890422
Epoch: 4120/20000, Loss: 0.0000367415268556
Epoch: 4130/20000, Loss: 0.0000353354698746
Epoch: 4140/20000, Loss: 0.0000344050349668
Epoch: 4150/20000, Loss: 0.0000341710874636
Epoch: 4160/20000, Loss: 0.0000338109784934
Epoch: 4170/20000, Loss: 0.0000335304539476
Epoch: 4180/20000, Loss: 0.0000332530507876
Epoch: 4190/20000, Loss: 0.0000329837712343
Epoch: 4200/20000, Loss: 0.0000327223497152
Epoch: 4210/20000, Loss: 0.0000328418063873
Epoch: 4220/20000, Loss: 0.0000520341054653
Epoch: 4230/20000, Loss: 0.0000363936887879
Epoch: 4240/20000, Loss: 0.0000351832240995
Epoch: 4250/20000, Loss: 0.00003

Epoch: 6020/20000, Loss: 0.0000115666598504
Epoch: 6030/20000, Loss: 0.0000117495483209
Epoch: 6040/20000, Loss: 0.0000129637819555
Epoch: 6050/20000, Loss: 0.0000190552491404
Epoch: 6060/20000, Loss: 0.0000119850174087
Epoch: 6070/20000, Loss: 0.0000118143316286
Epoch: 6080/20000, Loss: 0.0000103578822745
Epoch: 6090/20000, Loss: 0.0000104063356048
Epoch: 6100/20000, Loss: 0.0000103760712591
Epoch: 6110/20000, Loss: 0.0000140462570926
Epoch: 6120/20000, Loss: 0.0000461401723442
Epoch: 6130/20000, Loss: 0.0000171123992914
Epoch: 6140/20000, Loss: 0.0000126990435092
Epoch: 6150/20000, Loss: 0.0000104738892333
Epoch: 6160/20000, Loss: 0.0000101588429970
Epoch: 6170/20000, Loss: 0.0000100820670923
Epoch: 6180/20000, Loss: 0.0000113838659672
Epoch: 6190/20000, Loss: 0.0000229598244914
Epoch: 6200/20000, Loss: 0.0000119501282825
Epoch: 6210/20000, Loss: 0.0000101143405118
Epoch: 6220/20000, Loss: 0.0000097457223092
Epoch: 6230/20000, Loss: 0.0000098457885542
Epoch: 6240/20000, Loss: 0.00000

Epoch: 7910/20000, Loss: 0.0000083855411503
Epoch: 7920/20000, Loss: 0.0000079725577962
Epoch: 7930/20000, Loss: 0.0000077614740803
Epoch: 7940/20000, Loss: 0.0000076719852586
Epoch: 7950/20000, Loss: 0.0000076543037721
Epoch: 7960/20000, Loss: 0.0000076399364843
Epoch: 7970/20000, Loss: 0.0000076360283856
Epoch: 7980/20000, Loss: 0.0000079402861957
Epoch: 7990/20000, Loss: 0.0000239312503254
Epoch: 8000/20000, Loss: 0.0000170725670614
Epoch: 8010/20000, Loss: 0.0000090045805337
Epoch: 8020/20000, Loss: 0.0000089446884886
Epoch: 8030/20000, Loss: 0.0000081246198533
Epoch: 8040/20000, Loss: 0.0000076626947703
Epoch: 8050/20000, Loss: 0.0000078945140558
Epoch: 8060/20000, Loss: 0.0000149529514601
Epoch: 8070/20000, Loss: 0.0000115262128020
Epoch: 8080/20000, Loss: 0.0000089742188720
Epoch: 8090/20000, Loss: 0.0000086486279542
Epoch: 8100/20000, Loss: 0.0000080363133748
Epoch: 8110/20000, Loss: 0.0000077200793385
Epoch: 8120/20000, Loss: 0.0000077953427535
Epoch: 8130/20000, Loss: 0.00001

...

Epoch: 11900/20000, Loss: 0.0000028493586797
Epoch: 11910/20000, Loss: 0.0000049757049965
Epoch: 11920/20000, Loss: 0.0000037670133679
Epoch: 11930/20000, Loss: 0.0000033625574360
Epoch: 11940/20000, Loss: 0.0000030415785659
Epoch: 11950/20000, Loss: 0.0000027700016290
Epoch: 11960/20000, Loss: 0.0000026823415737
Epoch: 11970/20000, Loss: 0.0000031278611914
Epoch: 11980/20000, Loss: 0.0000344702093571
Epoch: 11990/20000, Loss: 0.0000106397346826
Epoch: 12000/20000, Loss: 0.0000049315153774
Epoch: 12010/20000, Loss: 0.0000030993126074
Epoch: 12020/20000, Loss: 0.0000028604540603
Epoch: 12030/20000, Loss: 0.0000028145859687
Epoch: 12040/20000, Loss: 0.0000057469683270
Epoch: 12050/20000, Loss: 0.0000086171694420
Epoch: 12060/20000, Loss: 0.0000059794165281
Epoch: 12070/20000, Loss: 0.0000065695980993
Epoch: 12080/20000, Loss: 0.0000045016417971
Epoch: 12090/20000, Loss: 0.0000028962842862
Epoch: 12100/20000, Loss: 0.0000031331471746
Epoch: 12110/20000, Loss: 0.0000051049719332
Epoch: 121

Epoch: 13770/20000, Loss: 0.0000024309151740
Epoch: 13780/20000, Loss: 0.0000051653796618
Epoch: 13790/20000, Loss: 0.0000076046089816
Epoch: 13800/20000, Loss: 0.0000108878894025
Epoch: 13810/20000, Loss: 0.0000052389063967
Epoch: 13820/20000, Loss: 0.0000028530575946
Epoch: 13830/20000, Loss: 0.0000024727337404
Epoch: 13840/20000, Loss: 0.0000034060801681
Epoch: 13850/20000, Loss: 0.0000057100410231
Epoch: 13860/20000, Loss: 0.0000063679985942
Epoch: 13870/20000, Loss: 0.0000020418042368
Epoch: 13880/20000, Loss: 0.0000021706905500
Epoch: 13890/20000, Loss: 0.0000018989034061
Epoch: 13900/20000, Loss: 0.0000030551341297
Epoch: 13910/20000, Loss: 0.0000086410955191
Epoch: 13920/20000, Loss: 0.0000079990550148
Epoch: 13930/20000, Loss: 0.0000082123378888
Epoch: 13940/20000, Loss: 0.0000028457363896
Epoch: 13950/20000, Loss: 0.0000021118180484
Epoch: 13960/20000, Loss: 0.0000018154918280
Epoch: 13970/20000, Loss: 0.0000020169961772
Epoch: 13980/20000, Loss: 0.0000046639793254
Epoch: 139

Epoch: 15650/20000, Loss: 0.0000017657703211
Epoch: 15660/20000, Loss: 0.0000013977472690
Epoch: 15670/20000, Loss: 0.0000013884020973
Epoch: 15680/20000, Loss: 0.0000024566204502
Epoch: 15690/20000, Loss: 0.0000067353521445
Epoch: 15700/20000, Loss: 0.0000086616109911
Epoch: 15710/20000, Loss: 0.0000062460135268
Epoch: 15720/20000, Loss: 0.0000022802839794
Epoch: 15730/20000, Loss: 0.0000018188842432
Epoch: 15740/20000, Loss: 0.0000016575380641
Epoch: 15750/20000, Loss: 0.0000016199610400
Epoch: 15760/20000, Loss: 0.0000041536327444
Epoch: 15770/20000, Loss: 0.0000064174519139
Epoch: 15780/20000, Loss: 0.0000090973644546
Epoch: 15790/20000, Loss: 0.0000050165258472
Epoch: 15800/20000, Loss: 0.0000028550214211
Epoch: 15810/20000, Loss: 0.0000017127713363
Epoch: 15820/20000, Loss: 0.0000015739577748
Epoch: 15830/20000, Loss: 0.0000015156044810
Epoch: 15840/20000, Loss: 0.0000067049118115
Epoch: 15850/20000, Loss: 0.0000042653887249
Epoch: 15860/20000, Loss: 0.0000072981265475
Epoch: 158

Epoch: 17520/20000, Loss: 0.0000016126093669
Epoch: 17530/20000, Loss: 0.0000017349266273
Epoch: 17540/20000, Loss: 0.0000037909721868
Epoch: 17550/20000, Loss: 0.0000028832544103
Epoch: 17560/20000, Loss: 0.0000030568185139
Epoch: 17570/20000, Loss: 0.0000112718880700
Epoch: 17580/20000, Loss: 0.0000045016809054
Epoch: 17590/20000, Loss: 0.0000026955747217
Epoch: 17600/20000, Loss: 0.0000014217227999
Epoch: 17610/20000, Loss: 0.0000011707114709
Epoch: 17620/20000, Loss: 0.0000013849511333
Epoch: 17630/20000, Loss: 0.0000074969534580
Epoch: 17640/20000, Loss: 0.0000084431994765
Epoch: 17650/20000, Loss: 0.0000031499569104
Epoch: 17660/20000, Loss: 0.0000013270049521
Epoch: 17670/20000, Loss: 0.0000010751857644
Epoch: 17680/20000, Loss: 0.0000011917300071
Epoch: 17690/20000, Loss: 0.0000044274293032
Epoch: 17700/20000, Loss: 0.0000061843707044
Epoch: 17710/20000, Loss: 0.0000045793062782
Epoch: 17720/20000, Loss: 0.0000020931670406
Epoch: 17730/20000, Loss: 0.0000018100195120
Epoch: 177

Epoch: 19350/20000, Loss: 0.0000027889198009
Epoch: 19360/20000, Loss: 0.0000042684732762
Epoch: 19370/20000, Loss: 0.0000018303080651
Epoch: 19380/20000, Loss: 0.0000011493191323
Epoch: 19390/20000, Loss: 0.0000009181378005
Epoch: 19400/20000, Loss: 0.0000013984010820
Epoch: 19410/20000, Loss: 0.0000060800111896
Epoch: 19420/20000, Loss: 0.0000034316826714
Epoch: 19430/20000, Loss: 0.0000097583142633
Epoch: 19440/20000, Loss: 0.0000028853994536
Epoch: 19450/20000, Loss: 0.0000013719596836
Epoch: 19460/20000, Loss: 0.0000009440599342
Epoch: 19470/20000, Loss: 0.0000006719911880
Epoch: 19480/20000, Loss: 0.0000005878994216
Epoch: 19490/20000, Loss: 0.0000008298667922
Epoch: 19500/20000, Loss: 0.0000097018182714
Epoch: 19510/20000, Loss: 0.0000052573063840
Epoch: 19520/20000, Loss: 0.0000026553161661
Epoch: 19530/20000, Loss: 0.0000008541821330
Epoch: 19540/20000, Loss: 0.0000014991303487
Epoch: 19550/20000, Loss: 0.0000073324026744
Epoch: 19560/20000, Loss: 0.0000016997441890
Epoch: 195

In [9]:
print(test_tensor.shape)
prediction_tensor = torch.zeros(1, 20, 256).float()
print(prediction_tensor.shape)

torch.Size([1, 1, 256])
torch.Size([1, 20, 256])


In [10]:
with torch.no_grad():
    prediction = lem(test_tensor)
    prediction = prediction.view(1, 1, 256).float()
    prediction_tensor[:, 0, :] = prediction
    for i in range(19):
        prediction = lem(prediction)
        prediction = prediction.view(1, 1, 256).float()
        prediction_tensor[:, i+1, :] = prediction
        

### Four different types of error

In [11]:
# Exact Solution

u_test = u_1.T
u_test_full = u_test[80:100, :]

In [12]:
prediction_tensor = torch.squeeze(prediction_tensor)


In [13]:
# Extrapolation

k1 = ( prediction_tensor - u_test_full)**2
u_test_full_tensor = torch.tensor(u_test_full**2)
u_test_full_tensor.shape

torch.Size([20, 256])

### L^2 norm error

In [14]:
# Compute the relative L2 error norm (generalization error)
relative_error_test = torch.mean(k1)/ torch.mean(u_test_full_tensor)

print("Relative Error Test: ", relative_error_test.item(), "%")

Relative Error Test:  0.05983429715206502 %


In [15]:
2+

SyntaxError: invalid syntax (4209523232.py, line 1)

### Max absolute norm error

In [ ]:
R_abs = torch.max(torch.abs(prediction_tensor - u_test_full))

In [ ]:
print(R_abs)

### Explained variance score

In [ ]:
import torch

a = prediction_tensor
b = u_test_full
# Assuming 'a' is your predicted values (model's predictions) and 'b' is the true values (ground truth)
# Make sure 'a' and 'b' are PyTorch tensors
# a = torch.tensor(a)
b = torch.tensor(b)
# Calculate the mean of 'b'
mean_b = torch.mean(b)

# Calculate the Explained Variance Score
numerator = torch.var(b - a)  # Variance of the differences between 'b' and 'a'
denominator = torch.var(b)    # Variance of 'b'
evs = 1 - numerator / denominator

print("Explained Variance Score:", evs.item())

### Mean absolute error

In [ ]:
# Compute the relative L2 error norm (generalization error)
relative_error_test = torch.mean(torch.abs(prediction_tensor - u_test_full))

print("Relative Error Test: ", relative_error_test, "%")

### Contour plot for PINN (80 percent) and (20 percentage lem prediction)

In [ ]:
print(prediction_tensor.shape)
prediction_tensor = torch.squeeze(prediction_tensor)
input_tensor = torch.squeeze(input_tensor)

conc_u = torch.squeeze(input_tensor)
concatenated_tensor = torch.cat((conc_u, prediction_tensor), dim=0)

x1 = np.linspace(-1, 1, 256)
t1 = np.linspace(0, 1, 99)

### Snapshot time plots

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

# Create the figure and axis objects with reduced width
fig, ax = plt.subplots(figsize=(5, 5))  # You can adjust the width (7 inches) and height (5 inches) as needed




final_time_output = prediction_tensor[3, :]
final_out = final_time_output.detach().numpy().reshape(-1, 1)
final_true = u_1[:, 83].reshape(-1, 1)

# Plot the data with red and blue lines, one with dotted and one with solid style
ax.plot(x, final_out, color='red', linestyle='dotted', linewidth=12, label='Prediction')
ax.plot(x, final_true, color='blue', linestyle='solid', linewidth=7, label='True')


# Set the axis labels with bold font weight
ax.set_xlabel(r"${x}$", fontsize=26, color='black', fontdict={'weight': 'bold'})
ax.set_ylabel(r"${u(x, t)}$", fontsize=26, color='black', fontdict={'weight': 'bold'})

# Set the title with bold font weight
ax.set_title(r"${t = 0.83}$", fontsize=26, color='black', fontweight='bold')

# Set the number of ticks for x-axis and y-axis to 3
ax.set_xticks([-1, 0, 1])
ax.set_yticks([-1, 0, 1])

# Set tick labels fontweight to bold and increase font size
ax.tick_params(axis='both', which='major', labelsize=20, width=2, length=10)

# # Set the fontweight for tick labels to bold
# for tick in ax.get_xticklabels() + ax.get_yticklabels():
#     tick.set_weight('bold')

# Set the spines linewidth to bold
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)

# Set the legend
# ax.legend()

plt.savefig('LEM_0.83_20.pdf', dpi=500, bbox_inches="tight")
#plt.savefig('lem_0.83_20.png', dpi=500, bbox_inches="tight")
# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

# Create the figure and axis objects with reduced width
fig, ax = plt.subplots(figsize=(5, 5))  # You can adjust the width (7 inches) and height (5 inches) as needed



final_time_output = prediction_tensor[-2, :]
final_out = final_time_output.detach().numpy().reshape(-1, 1)
final_true = u_1[:, -2].reshape(-1, 1)

# Plot the data with red and blue lines, one with dotted and one with solid style
ax.plot(x, final_out, color='red', linestyle='dotted', linewidth=12, label='Prediction')
ax.plot(x, final_true, color='blue', linestyle='solid', linewidth=7, label='True')


# Set the axis labels with bold font weight
ax.set_xlabel(r"${x}$", fontsize=26, color='black', fontdict={'weight': 'bold'})
ax.set_ylabel(r"${u(x, t)}$", fontsize=26, color='black', fontdict={'weight': 'bold'})

# Set the title with bold font weight
ax.set_title(r"${t = 0.98}$", fontsize=26, color='black', fontweight='bold')

# Set the number of ticks for x-axis and y-axis to 3
ax.set_xticks([-1, 0, 1])
ax.set_yticks([-1, 0, 1])

# Set tick labels fontweight to bold and increase font size
ax.tick_params(axis='both', which='major', labelsize=20, width=2, length=10)

# # Set the fontweight for tick labels to bold
# for tick in ax.get_xticklabels() + ax.get_yticklabels():
#     tick.set_weight('bold')

# Set the spines linewidth to bold
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)

# Set the legend
# ax.legend()

plt.savefig('LEM_0.98_20.pdf', dpi=500, bbox_inches="tight")
#plt.savefig('lem_0.98_20.png', dpi=500, bbox_inches="tight")
# Show the plot
plt.show()


### Contour plot where 80 percent for PINN solution and 20 percent for lem solution

### Exact contour

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator

# Assuming you have defined concatenated_tensor as a PyTorch tensor
# concatenated_tensor = torch.cat((tensor1, tensor2), dim=0)

# Convert concatenated_tensor to a NumPy array
concatenated_array = u_1.T

# Define custom color levels
x = np.linspace(-1, 1, concatenated_array.shape[1])  # Replace 0 and 1 with your actual x range
t = np.linspace(0, 1, concatenated_array.shape[0])  # Replace 0 and 1 with your actual t range
X, T = np.meshgrid(x, t)

# Define custom color levels using the minimum and maximum from the NumPy array
c_levels = np.linspace(np.min(concatenated_array), np.max(concatenated_array), 400)

# Plot the contour with interpolated data
plt.figure(figsize=(20, 5))
plt.pcolormesh(T, X, concatenated_array, shading='auto', cmap='coolwarm')

# Set the fontweight for axis labels to regular (not bold)
plt.xlabel("$t$", fontsize=26)
plt.ylabel("$x$", fontsize=26)
plt.title("$u(x, t)$", fontsize=26)

# Set tick labels fontweight to regular (not bold) and increase font size
plt.tick_params(axis='both', which='major', labelsize=20, width=3, length=10)

# Set the fontweight for tick labels to regular (not bold)
for tick in plt.gca().get_xticklabels() + plt.gca().get_yticklabels():
    tick.set_weight('normal')

# Set the number of ticks for x-axis and y-axis to 5
num_ticks = 5
x_ticks = np.linspace(np.min(T), np.max(T), num_ticks)
y_ticks = np.linspace(np.min(X), np.max(X), num_ticks)

plt.gca().xaxis.set_major_locator(FixedLocator(x_ticks))
plt.gca().yaxis.set_major_locator(FixedLocator(y_ticks))

cbar1 = plt.colorbar()
# Set the number of ticks for the color bar with uniformly distributed numbers
num_ticks = 5
c_ticks = np.linspace(np.min(concatenated_array), np.max(concatenated_array), num_ticks)
cbar1.set_ticks(c_ticks)

# Set the fontweight and fontsize for color bar tick labels
for t in cbar1.ax.get_yticklabels():
    t.set_weight('normal')
    t.set_fontsize(26)  # Increase the font size for color bar tick labels

# Increase the size of numbers on axis and color bar
plt.xticks(fontsize=26)
plt.yticks(fontsize=26)

# Increase the tick size and width of the color bar
cbar1.ax.tick_params(axis='both', which='major', labelsize=30, width=3,  length=10)

#plt.savefig('Contour_Exact.pdf', dpi=500, bbox_inches="tight")
plt.savefig('contour_exact.jpeg', dpi=500, bbox_inches="tight")
# Show the plot
plt.show()


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator

# Assuming you have defined concatenated_tensor as a PyTorch tensor
# concatenated_tensor = torch.cat((tensor1, tensor2), dim=0)

# Convert concatenated_tensor to a NumPy array
concatenated_array = concatenated_tensor.numpy()

# Define custom color levels
x = np.linspace(-1, 1, concatenated_array.shape[1])  # Replace 0 and 1 with your actual x range
t = np.linspace(0, 1, concatenated_array.shape[0])  # Replace 0 and 1 with your actual t range
X, T = np.meshgrid(x, t1)

# Define custom color levels using the minimum and maximum from the NumPy array
c_levels = np.linspace(np.min(concatenated_array), np.max(concatenated_array), 400)

# Plot the contour with interpolated data
plt.figure(figsize=(20, 5))
plt.pcolormesh(T, X, concatenated_array, shading='auto', cmap='coolwarm')

# Set the fontweight for axis labels to regular (not bold)
plt.xlabel("$t$", fontsize=26)
plt.ylabel("$x$", fontsize=26)
plt.title("$u(x, t)$", fontsize=26)

# Set tick labels fontweight to regular (not bold) and increase font size
plt.tick_params(axis='both', which='major', labelsize=20, width=3, length=10)

# Set the fontweight for tick labels to regular (not bold)
for tick in plt.gca().get_xticklabels() + plt.gca().get_yticklabels():
    tick.set_weight('normal')

# Set the number of ticks for x-axis and y-axis to 5
num_ticks = 5
x_ticks = np.linspace(np.min(T), np.max(T), num_ticks)
y_ticks = np.linspace(np.min(X), np.max(X), num_ticks)

plt.gca().xaxis.set_major_locator(FixedLocator(x_ticks))
plt.gca().yaxis.set_major_locator(FixedLocator(y_ticks))

cbar1 = plt.colorbar()
# Set the number of ticks for the color bar with uniformly distributed numbers
num_ticks = 5
c_ticks = np.linspace(np.min(concatenated_array), np.max(concatenated_array), num_ticks)
cbar1.set_ticks(c_ticks)

# Set the fontweight and fontsize for color bar tick labels
for t in cbar1.ax.get_yticklabels():
    t.set_weight('normal')
    t.set_fontsize(26)  # Increase the font size for color bar tick labels

# Increase the size of numbers on axis and color bar
plt.xticks(fontsize=26)
plt.yticks(fontsize=26)

# Increase the tick size and width of the color bar
cbar1.ax.tick_params(axis='both', which='major', labelsize=30, width=3,  length=10)

# Add a dotted line at t = 0.8
plt.axvline(x=0.8, color='black', linestyle='dotted', linewidth=5)

#plt.savefig('Contour_LEM_20.pdf', dpi=500, bbox_inches="tight")
plt.savefig('contour_LEM_20.jpeg', dpi=500, bbox_inches="tight")
# Show the plot
plt.show()
